In [ ]:
import pandas as pd
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

# Define SVM tool
def predict_stroke_svm(input_data: dict) -> str:
    import joblib
    from sklearn.preprocessing import LabelEncoder

    # Load encoders
    mapping = {
        'gender': ['Female', 'Male', 'Other'],
        'ever_married': ['No', 'Yes'],
        'work_type': ['Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'],
        'Residence_type': ['Rural', 'Urban'],
        'smoking_status': ['Unknown', 'formerly smoked', 'never smoked', 'smokes']
    }

    encoders = {}
    for col, values in mapping.items():
        le = LabelEncoder()
        le.fit(values)
        encoders[col] = le

    model = joblib.load("svm_stroke_model.joblib")
    scaler = joblib.load("svm_stroke_scaler.joblib")

    encoded_input = {}
    for key in input_data:
        if key in encoders:
            encoded_input[key] = encoders[key].transform([input_data[key]])[0]
        else:
            encoded_input[key] = input_data[key]

    feature_order = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
                     'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']

    X_input = pd.DataFrame([encoded_input], columns=feature_order)
    X_scaled = scaler.transform(X_input)


    pred = model.predict(X_scaled)[0]
    prob = model.predict_proba(X_scaled)[0][int(pred)]
    return f"Prediction: {pred}, Probability: {prob:.2f}"

#  Tool calling structure
stroke_tool = Tool(
    name="StrokeSVM",
    func=lambda x: predict_stroke_svm(eval(x)),  
    description="Use this tool to predict stroke risk. Input must be a dictionary with keys: gender, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status"
)

# Initialize LLaMA 3.1 LLM via OllamaLLM
llm = OllamaLLM(model="llama3.1:latest")

# Create ReAct agent
agent = initialize_agent(
    tools=[stroke_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Ask the question
query = '''
Predict stroke risk for a 67 year old male who is married, works in Private sector, has heart disease,
does not have hypertension, smokes, and has glucose level 228.69 and BMI 36.6.

Use dictionary format like:
{"gender": "Male", "age": 67, "hypertension": 0, "heart_disease": 1,
 "ever_married": "Yes", "work_type": "Private", "Residence_type": "Urban",
 "avg_glucose_level": 228.69, "bmi": 36.6, "smoking_status": "formerly smoked"}
'''

agent.invoke(query)



> Entering new AgentExecutor chain...
Action: StrokeSVM
Action Input: {"gender": "Male", "age": 67, "hypertension": 0, "heart_disease": 1, 
"ever_married": "Yes", "work_type": "Private", "Residence_type": "Urban",
 "avg_glucose_level": 228.69, "bmi": 36.6, "smoking_status": "formerly smoked"}
Observation: Prediction: 0, Probability: 0.94
Thought:Question:
Predict stroke risk for a 67 year old male who is married, works in Private sector, has heart disease,
does not have hypertension, smokes, and has glucose level 228.69 and BMI 36.6.

Use dictionary format like:
{"gender": "Male", "age": 67, "hypertension": 0, "heart_disease": 1,
 "ever_married": "Yes", "work_type": "Private", "Residence_type": "Urban",
 "avg_glucose_level": 228.69, "bmi": 36.6, "smoking_status": "formerly smoked"}

Thought: The input data seems correct, so I will use the StrokeSVM tool.
Action: StrokeSVM
Action Input: {"gender": "Male", "age": 67, "hypertension": 0, "heart_disease": 1,
"ever_married": "Yes", "work_t

{'input': '\nPredict stroke risk for a 67 year old male who is married, works in Private sector, has heart disease,\ndoes not have hypertension, smokes, and has glucose level 228.69 and BMI 36.6.\n\nUse dictionary format like:\n{"gender": "Male", "age": 67, "hypertension": 0, "heart_disease": 1,\n "ever_married": "Yes", "work_type": "Private", "Residence_type": "Urban",\n "avg_glucose_level": 228.69, "bmi": 36.6, "smoking_status": "formerly smoked"}\n',
 'output': 'The predicted stroke risk is 0 with a probability of 0.94.'}